# PySCIPOpt Tutorial: How to write your own plug-ins

In this notebook you will find an introduction to writing custom plug-ins for SCIP using the Python interface (PySCIPOpt). In particular, I will show, through examples, how to write custom branching rules and node selection rules.

I will assume a basic understanding of the branch-and-bound algorithm and other common techniques to tackle Mixed Integer Programs (MIPs).

&nbsp;
<br />
&nbsp;
<br />

# 1. Reading and solving your instance

Let's start by reading an instance from an .lp file and solving it. The following code will do so by using all default SCIP settings. 

In [28]:
import pyscipopt as scip
import numpy as np
import random

filename = 'instance.lp'

model = scip.Model()
model.readProblem(filename)
model.optimize()
model.freeProb()

SCIP has a great deal of parameters. You can find a complete list [here](https://www.scipopt.org/doc-7.0.0/html/PARAMETERS.php).

When using the python interface, there is an easy way to set the parameters to something different to the default values. In this case, let's set the verbosity level to zero.

In [ ]:
model = scip.Model()
model.readProblem(filename)
model.setParam('display/verblevel', 0)
model.optimize()
model.freeProb()

&nbsp;
<br />
&nbsp;
<br />

# 2. How to create a custom branching rule

Branching rules dictate how the search space is partitioned. The most common approach is to choose from the set of integer variables with a fractional value in the current LP-relaxation solution, to then partition the space by restricting the bounds of this variable.

This is, imagine we process a node and obtain a solution $\tilde{x}$ to the LP-relaxation. Say that variable $j$ (which is an integer variable) takes value $\tilde{x}_j=2.6$. This obviously violates the integrality constraints. Therefore, we can create two child nodes by imposing $x_j\leq 2$ on the left one and $x_j\geq 3$ on the right one.

SCIP provides a variety of already implemented branching rules (more information about them [here](https://www.scipopt.org/doc-7.0.1/html/group__BRANCHINGRULES.php)). The solver chooses among the rule according to their priority settings. These are the default values for the priorities:

|        Name       	|      Priority      	|                                                     Description                                                    	|
|:-----------------:	|:------------------:	|:------------------------------------------------------------------------------------------------------------------:	|
|     Allfullstrong 	|       $-1000$      	| All variable full strong branching                                                                                 	|
|             Cloud 	|         $0$        	| Cloud branching ([paper](http://webdoc.sub.gwdg.de/ebook/serien/ah/ZIB/ZR-13-01.pdf))                              	|
|      Distribution 	|         $0$        	| Probability based branching rule ([paper](http://www.sce.carleton.ca/faculty/chinneck/docs/PryorChinneck.pdf))     	|
|        Fullstrong 	|         $0$        	| Full strong branching                                                                                              	|
|          Leastinf 	|        $50$        	| Least infeasible rule                                                                                              	|
|         Lookahead 	|         $0$        	| Lookahead branching ([paper](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.444.8532&rep=rep1&type=pdf)) 	|
|           Mostinf 	|        $100$       	| Most infeasible rule                                                                                               	|
|          Multaggr 	|         $0$        	| Fullstrong branching on fractional and multi-aggregated variables                                                  	|
|            Pscost 	|       $2000$       	| Pseudocosts branching rule                                                                                         	|
|            Random 	|       $-10^5$      	| Random branching rule                                                                                              	|
|         Relpscost 	|       $10000$      	| Reliability pseudocosts rule ([paper](http://miplib2010.zib.de/miplib2003/paper/achterbergkochmartin2005_pp.pdf))  	|
| Vanillafullstrong 	|       $-2000$      	| Vanilla full strong branching.                                                                                     	|

In order to create a custom branching rule in SCIP, we need to create a class that inherits from `scip.Branchrule`. Let's look at the following example of a branching rule that takes decisions at random.

In [3]:
class RandomBranchRule(scip.Branchrule):
    def __init__(self):
        pass

    def branchinit(self):
        ''' executed after the problem has been transformed. '''
        pass
    
    def branchinitsol(self):
        '''executed when the presolving is finished and the branch-and-bound process is about to begin. '''
        pass

    def branchexeclp(self, allowaddcons):
        # Executed during node processing if a fractional LP solution is available.
        candidate_vars, *_ = self.model.getLPBranchCands()
        branch_var_idx = random.randint(0,len(candidate_vars)-1)
        branch_var = candidate_vars[branch_var_idx]
        self.model.branchVar(branch_var)
        result = scip.SCIP_RESULT.BRANCHED
        return {"result": result}

    def branchexitsol(self):
        '''executed before the branch-and-bound process is freed '''
        pass

    def branchexit(self):
        '''executed before the transformed problem is freed'''
        pass
    
    def branchfree(self):
        '''frees memory of branching rule'''
        pass

Clearly most of the methods of this class are not doing anything, but I have included them so you can use this as a template. The most important method is `branchexeclp()`. Here we pass SCIP the variable that the solver will use to branch.

The next step is to include our custom branching rule and solve the instance.

In [ ]:
model = scip.Model()
model.readProblem(filename)
branchrule = RandomBranchRule()
model.includeBranchrule(branchrule=branchrule,
                        name="CustomRand", # name of the branching rule
                        desc="",           # description of the branching rule
                        priority=100000,   # priority: set to this to make it default
                        maxdepth=-1,       # maximum depth up to which it will be used, or -1 for no restriction
                        maxbounddist=1)    # maximal relative distance from current node's dual bound to primal 
                                           # bound compared to best node's dual bound for applying branching rule
                                           # (0.0: only on current best node, 1.0: on all nodes)
model.optimize()
model.freeProb()

**Note**: this is clearly a very dumb rule. Notice how much longer the solver takes to solve the problem!

Let's look now at another (slightly more advanced) example. 

In [18]:
class MostInfBranchRule(scip.Branchrule):

    def branchexeclp(self, allowaddcons):
        # Executed during node processing if a fractional LP solution is available.
        candidate_vars, candidate_vals, candidate_fracs, *_ = self.model.getLPBranchCands()
        branch_var_idx = np.argmax(candidate_fracs)
        branch_var = candidate_vars[branch_var_idx]
        self.model.branchVar(branch_var)
        result = scip.SCIP_RESULT.BRANCHED
        return {"result": result}


The two examples that I have presented here are quite bad rules for branching. You can construct fancier rules and extract more information about the candidates using the multiple callbacks that SCIP provides.

**Note**: Not all C callbacks are implemented in PySCIPOpt, but you can easily extend the interface using function wrappers!

&nbsp;
<br />
&nbsp;
<br />

# 3. How to create a custom node selection rule.

Node selection rules are responsible for choosing the subproblem that will be processed next. The most widely accepted rules usually balance two strategies: (i) choosing a child of the current node (and therefore going deep into the tree, a.k.a. plunging) and (ii) choosing nodes with good lower bounds. For this reason, SCIP stores children, siblings and leaves in separate data structures. In this context, "leaves" refers to all open nodes that are not children or siblings of the current node.

These are the node selection rules that SCIP offers (more info [here](https://www.scipopt.org/doc-7.0.1/html/group__NODESELECTORS.php)): 

| Name        	| Standard Priority 	| Memory save priority 	| Description                                                                                         	|
|-------------	|-------------------	|----------------------	|-----------------------------------------------------------------------------------------------------	|
| BFS         	|      100,000      	|           0          	| Best first search                                                                                   	|
| Breadth     	|      -10,000      	|      -1,000,000      	| Breadth first search                                                                                	|
| DFS         	|         0         	|        100,000       	| Depth first search                                                                                  	|
| Estimate    	|      200,000      	|          100         	| Best estimate search                                                                                	|
| Hybrid      	|       50,000      	|          50          	| Hybrid best estimate - best first                                                                   	|
| Restart DFS 	|       10,000      	|        50,000        	| Restart depth first search                                                                          	|
| UCT         	|         10        	|           0          	| Multi-armed bandit based rule ([paper](http://www.cs.toronto.edu/~horst/cogrobo/papers/uctmip.pdf)) 	|

Just like with the brancher plug-in, for node selection we need to define a node selector class.
Here, you will be working with node objects. You can find the definition of the Node class in `PySCIPOpt/src/pyscipopt/scip.pyx`. I recommend you to take a look at the different methods that you can use when building your rule. Here I will present some examples.

Our node selector class has two fundamental methods:
- `nodecomp(node1,node2)`: called to compare two leaves according to your chosen criterion. The inputs are two node objects and the output is a float indicating the preference. 
- `nodeselect()`: called at each iteration of the solving loop. Here is where the actual node selection takes place. You must return a dictionary of the type `{’selnode’: node}` where node is a node object.

Here is an example of the depth-first search rule.

In [23]:
class DFSNodeSelector(scip.Nodesel):
    def __init__(self):
        pass
    
    def nodeinit(self):
        ''' executed after the problem is transformed. use this call to initialize node selector data.'''
        pass
    
    def nodeinitsol(self):
        '''executed when the presolving is finished and the branch-and-bound process is about to begin'''
        pass

    def nodeselect(self):
        '''first method called in each iteration in the main solving loop. '''
        leaves, children, siblings = self.model.getOpenNodes()
        
        # Select one of the children, if any.
        if len(children)>0:
            selnode = children[0]
        # If there are no children in the list currently, check for siblings.
        elif len(siblings)>0:
            selnode = siblings[0]
        # Finally, if there are no children and no siblings, get the leaf with highest priority.
        elif len(leaves)>0:
            selnode = self.model.getBestLeaf()
        # If the open node list is empty, pass None to terminate search.
        else:
            selnode = None
        return {"selnode": selnode}

    def nodecomp(self, node1, node2):
        '''
        compare two leaves of the current branching tree
        It should return the following values:
          value < 0, if node 1 comes before (is better than) node 2
          value = 0, if both nodes are equally good
          value > 0, if node 1 comes after (is worse than) node 2.
        '''
        d1 = node1.getDepth()
        d2 = node2.getDepth()
        value = d2 - d1
        return value
    
    def nodeexitsol(self):
        '''executed before the branch-and-bound process is freed'''
        pass    

    def nodeexit(self):
        '''executed before the transformed problem is freed'''
        pass
    
    def nodefree(self):
        '''frees memory of node selector'''
        pass


Once again I have included all possible methods for our class, so you can use it as a template, but I will be omitting the unnecessary ones from now on.

With this example I wanted to demonstrate the use of `getOpenNodes()`. Observe that, as I mentioned before, SCIP will subdivide the list of open nodes, because it keeps track of the children, siblings and the rest of the leaves separately. This can be very useful at times.

We can write the DFS in a cleaner way by using the SCIP callbacks that return the best node (from each subclass) according to `nodecomp()`.  If the category is empty, this callback returns `None`. 

In [25]:
class DFSNodeSelector_alt(scip.Nodesel):
    def __init__(self):
        pass

    def nodeselect(self):
        '''first method called in each iteration in the main solving loop. '''
        selnode = self.model.getBestChild()
        if selnode is None:
            selnode = self.model.getBestSibling()
        if selnode is None:
            selnode = self.model.getBestLeaf()
        return {"selnode": selnode}

    def nodecomp(self, node1, node2):
        '''
        compare two leaves of the current branching tree
        It should return the following values:
          value < 0, if node 1 comes before (is better than) node 2
          value = 0, if both nodes are equally good
          value > 0, if node 1 comes after (is worse than) node 2.
        '''
        d1 = node1.getDepth()
        d2 = node2.getDepth()
        value = d2 - d1
        return value


We can also obtain the best node across all node subclasses using `getBestNode()`. In the case of DFS, however, doing this would be slightly less efficient, given that we know that children always have priority. By distinguishing node categories, we spare some comparisons.

This is not true for other node selection rules though. Here is an example of a best first rule:

In [27]:
class BFSNodeSelector(scip.Nodesel):
    
    def nodeselect(self):
        '''first method called in each iteration in the main solving loop. '''
        selnode = self.model.getBestNode()
        return {"selnode": selnode}

    def nodecomp(self, node1, node2):
        '''
        compare two leaves of the current branching tree
        It should return the following values:
          value < 0, if node 1 comes before (is better than) node 2
          value = 0, if both nodes are equally good
          value > 0, if node 1 comes after (is worse than) node 2.
        '''
        b1 = node1.getLowerbound()
        b2 = node2.getLowerbound()
        if self.model.isLT(b1,b2):
            value = -1
        elif self.model.isGT(b1,b2):
            value = 1
        else:
            value = 0
        return value


**Note 1**: This is a vanilla implementation of best first search. The actual implementation of SCIP is more sophisticated (e.g. performs plunging rounds).

**Note 2**: We have seen examples of the usage of `getBestChild()`. This method returns the best child according to `nodecomp()`. There is another option available: `getPrioChild()`. The difference is that this "priority child" is selected according to the branching rule. Not all branching rules implement a priority child selection. The same applies for the sibling category (i.e. `getBestSibling()` vs `getPrioSibling()`). Unfortunately, as of the day this tutorial was written, PySCIPOpt has no wrapper for the `SCIPgetPrioChild()` callback, but you can easily implement this is you need it.

Finally, you only have to include your selection rule to the model, in the following way:

In [ ]:
model = scip.Model()
model.setIntParam('display/verblevel', 0)
model.readProblem(filename)
nodeselrule = DFSNodeSelector_alt()
model.includeNodesel(nodeselrule,
                     name="custom_dfs",      # name of node selector
                     desc="",                # description of node selector
                     stdpriority=300000,     # priority of the node selector in standard mode
                     memsavepriority=300000) # priority of the node selector in memory saving mode
model.optimize()
model.freeProb()

A final comment: notice how much more the solving time deteriorates if we choose a "dumb" branching rule, compared to choosing a "dumb" node selection rule!